In [0]:
from zipfile import ZipFile
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Reading the zip file and storint it as an archive
train_imgArchive = ZipFile('/content/drive/My Drive/ComputerVision/train.zip', 'r')

In [0]:
y_train = []
x_train = []

#Get the list of info on all the files/folders preset in the archive
file_list = train_imgArchive.infolist()
for fle in file_list:

  #For each entry in the archive, check if it is file or folder and if it is a file then proceed to read the file. 
  if fle.is_dir() is False:
    with train_imgArchive.open(fle) as file:
            img = Image.open(file)
            #PIL image is saved as an png image, to convert it in to an array, we use np.array(image)
            x_train.append(np.array(img))
    #Add the corresponding class of the image to the target variable.
    img_info = fle.filename.split('/')
    img_class = img_info[1]
    y_train.append(img_class)

In [0]:
x_test = []

#Reading the zip file and storint it as an archive
test_imgArchive = ZipFile('/content/drive/My Drive/ComputerVision/test.zip', 'r')

test_file_list = test_imgArchive.infolist()
for fle in test_file_list:

  #For each entry in the archive, check if it is file or folder and if it is a file then proceed to read the file. 
  if fle.is_dir() is False:
    with test_imgArchive.open(fle) as file:
            img = Image.open(file)
            #PIL image is saved as an png image, to convert it in to an array, we use np.array(image)
            x_test.append(np.array(img))


In [0]:
for i in range(len(x_train)):
  x_train[i].resize(64,64,3)

In [0]:
for i in range(len(x_test)):
  x_test[i].resize(64,64,3)

In [0]:
X_train = np.stack(x_train)

In [0]:
X_test = np.stack(x_test)

In [0]:
X_train.shape

(4750, 64, 64, 3)

In [0]:
X_test.shape

(794, 64, 64, 3)

In [0]:
y_train = np.array(y_train)

In [0]:
y_train.shape

(4750,)

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D, Convolution3D, MaxPooling3D
from keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [0]:
print(np.unique(y_train))
np.unique(y_train).size

12

In [0]:
le = LabelEncoder()
Y_train = le.fit_transform(y_train)

In [0]:
Y_train = np_utils.to_categorical(Y_train)

In [0]:
Y_train.shape

(4750, 12)

In [0]:
from sklearn.model_selection import train_test_split
X_training, X_val, y_training, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=50)
print(X_training.shape)
print(y_training.shape)
print(X_val.shape)
print(y_val.shape)

(3800, 64, 64, 3)
(3800, 12)
(950, 64, 64, 3)
(950, 12)


In [0]:
#to check for convolution3d
X_train_nn = X_training.reshape(X_training.shape[0], 64, 64, 3, 1).astype('float32')
X_val_nn = X_val.reshape(X_val.shape[0], 64, 64, 3, 1).astype('float32')

In [0]:
BATCH_SIZE = 64
EPOCHS = 10

In [0]:
model = Sequential()

model.add(BatchNormalization(input_shape = (64,64,3)))
model.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (64, 64, 3))) 
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))

model.add(Convolution2D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))

model.add(Convolution2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=64,activation = 'relu'))

model.add(Dense(units = 128, activation = 'relu'))

model.add(Dropout(0.5))
model.add(Dense(units = 32, activation = 'relu'))

model.add(Dropout(0.2))
model.add(Dense(units = 12, activation = 'softmax')) 













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
optimizer = optimizers.Adam(lr=0.003)
model.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics = ['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=7, verbose=1, mode='auto')
callback_list = [early_stopping]

In [0]:
# Train the model
model.fit(X_training, y_training, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, callbacks=callback_list, validation_data=(X_val, y_val))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.




Train on 3800 samples, validate on 950 samples
Epoch 1/10
3800/3800 [==============================] - 59s 16ms/step - loss: 2.4689 - acc: 0.1213 - val_loss: 2.4346 - val_acc: 0.1400
Epoch 2/10
3800/3800 [==============================] - 58s 15ms/step - loss: 2.4178 - acc: 0.1450 - val_loss: 2.3822 - val_acc: 0.1842
Epoch 3/10
3800/3800 [==============================] - 58s 15ms/step - loss: 2.3449 - acc: 0.1908 - val_loss: 2.3070 - val_acc: 0.2000
Epoch 4/10
3800/3800 [==============================] - 58s 15ms/step - loss: 2.3001 - acc: 0.1997 - val_loss: 2.3022 - val_acc: 0.2021
Epoch 5/10
3800/3800 [==============================] - 59s 16ms/step - loss: 2.2759 - acc: 0.2074 - val_loss: 2.3285 - val_acc: 0.2126
Epoch 6/10
3800/3800 [==============================] - 59s 16ms/step - loss: 2.2822 - acc: 0.2032 - val_loss: 2.2791 - val_acc: 0.2158
Epoch 7/10
3800/3800 [==============================] - 59s 16ms/step - loss: 2.2808 - acc: 0.2082 - val_loss: 2.2993 - val_acc: 0.2105